<a href="https://colab.research.google.com/github/xhaktmchl/SAI_2020-2_keras_study/blob/main/SAI_keras_3_4%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###3.4영화 리뷰 분류

###3.4.1 데이터 불러오기

In [ ]:
import keras 

In [ ]:
from keras.datasets import imdb

(train_data,train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000) 
# num_words=10000 자주 나타나는 단어 10000개 뽑기
# data= 리뷰, labels = 0부정, 1 긍정 

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
train_data
train_labels[0] 

1

###3.4.2 데이터 준비

신경망에 숫자 리스트 불가능=> 리스트를 텐서로 변환  

1) 리스트에 패딩 추가해서 (samples, sequence_length) 크기의 정수 텐서로 변환  
2) 원-핫 인코딩하여 0과 1의 벡터로 변환



In [ ]:
# 숫자리스트를 벡터로 변환하기
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듦
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results

    # for i,v in enumerate() : 인덱스와 원소값 같이 튜플(,) 형태로 나옴

   # 훈련 데이터를 벡터로 변환
x_train = vectorize_sequences(train_data)
   # 테스트 데이터를 벡터로 변환
x_test = vectorize_sequences(test_data)
  


In [ ]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [ ]:
# 레이블을 벡터로 바꿈
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

###3.4.3 신경망 모델 만들기

- Dense 층에 전달한 매개변수(16)는 은닉 유닛의 개수입니다. 하나의 은닉 유닛은 층이 나타내는 표현 공간에서 하나의 차원이 됩니다.     

- 16개의 은닉 유닛이 있다는 것은 가중치 행렬 W의 크기가 (input_dimension, 16)이라는 뜻

- 은닉 유닛의 개수를 늘리면 정확도 증가 , 하지만 계산 비용이 커지고 원치 않은 패턴을 학습할 수 있음( 테스트 데이터에선 정확도 떨어질 수)  

Dense 층을 쌓을 때 두 가진 중요한 구조상의 결정이 필요하다:

- 얼마나 많은 층을 사용할 것인가  
- 각 층에 얼마나 많은 은닉 유닛을 둘 것인가  

보통 relu 함수 이용하지만
여기선 감정을 0,1로 나타내서 시그모이드 함수 이용


1) 신경말 모델  
2개의 층(relu함수)- 마지막 (시그모이드 함수)=>0,1 출력


In [ ]:
# 모델 층 정의
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,))) #더한다 dense 층을 은닉유닛,relu함수,10000행 # //Q) 여기선 오류 안난다 ;; 뭐가 문제지
model.add(layers.Dense(16, activation='relu')) 
model.add(layers.Dense(1, activation='sigmoid'))

다음은 모델정의

In [ ]:
# 기본 모델 정의
model.compile(optimizer='rmsprop', # 옵티마이저
              loss='binary_crossentropy', # 확률 분포간의 차이를 측정
              metrics=['accuracy']) # 정확도 지표

옵티마이저의 매개변수 바꾸기 표현( Q) 아직은 잘 모르겠다.)

In [ ]:
# 모델 정의1(옵티마이저 매개 변수 직접 )
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001), # 객체 만들어 전달
              loss='binary_crossentropy',
              metrics=['accuracy'])


자신만의 손실함수, 측정함수 전달 표현( 표현이 있다는 것 정도만)

In [ ]:
# 모델정의 2 손실과 측정을  객체로 만들는 방법
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy, 
              metrics=[metrics.binary_accuracy])

### 훈련 검증

모델정의 했으니 fit으로 훈련반복

  하기 전 샘플을 가져와 모델을 검증한다

In [ ]:
# 샘플로 모델 검증
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
# fit 훈련 반복 ( 512개 샘플씩 20번 반복)
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))



Epoch 1/20
30/30 [==============================] - 1s 33ms/step - loss: 0.5463 - binary_accuracy: 0.7630 - val_loss: 0.4160 - val_binary_accuracy: 0.8657
Epoch 2/20
30/30 [==============================] - 1s 20ms/step - loss: 0.3288 - binary_accuracy: 0.8971 - val_loss: 0.3234 - val_binary_accuracy: 0.8796
Epoch 3/20
30/30 [==============================] - 1s 19ms/step - loss: 0.2322 - binary_accuracy: 0.9283 - val_loss: 0.2840 - val_binary_accuracy: 0.8882
Epoch 4/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1866 - binary_accuracy: 0.9404 - val_loss: 0.2855 - val_binary_accuracy: 0.8848
Epoch 5/20
30/30 [==============================] - 1s 20ms/step - loss: 0.1466 - binary_accuracy: 0.9550 - val_loss: 0.2790 - val_binary_accuracy: 0.8882
Epoch 6/20
30/30 [==============================] - 1s 20ms/step - loss: 0.1242 - binary_accuracy: 0.9618 - val_loss: 0.2895 - val_binary_accuracy: 0.8848
Epoch 7/20
30/30 [==============================] - 1s 20ms/step - los

model.fit() 메서드는 History 객체를 반환. history 속성은 모든 정보를 포함

In [ ]:
history_dict = history.history
history_dict.keys()


dict_keys(['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy'])

###matplotlib 시각화



In [ ]:
import matplotlib.pyplot as plt

1.loss 함수의 시각화

In [ ]:
acc = history.history['acc'] # history 객체에서 data 정확도 저장 //Q) acc 오류남 업데이트 되서 다른 버전인듯
val_acc = history.history['val_acc'] # history 에서 label 정확도 저장
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# 매틀랩이랑 비슷
# ‘bo’는 파란색 점을 의미합니다
#  x,y축 과 이름 달기
plt.plot(epochs, loss, 'bo', label='Training loss') 
# ‘b’는 파란색 실선을 의미합니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

KeyError: ignored

2. 정확도(acc)의 시각화

In [ ]:
plt.clf()   # 그래프를 초기화합니다
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


### 그래프의 해석
각 표의 점선은 훈련 손실과 훈련 정확도이고   
실선은 훈련검증의 손실과 정확도이다.

훈련 손실은 최소화되고 정확도는 점점 증가하므로 훈련데이터는 잘 학습.   
하지만 검증 손실은 증가하고 검증 정혹도는 낮아지므로 훈련데이터에서만 잘 학습  => 오버피팅(과대적합) 

### 과대적합 해결

: 모델 초기화 후 4번의 에포크만 학습 

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', # 최적화
              loss='binary_crossentropy', # 크로스 엔트로피 손실함수 (확률을 구할때 효과적)
              metrics=['accuracy']) 

model.fit(x_train, y_train, epochs=4, batch_size=512) # 4번만 학습
results = model.evaluate(x_test, y_test) # 모델 평가

In [ ]:
# 결과 출력
results 

결과 : 모델정확도 증가

### 훈련된 모델로 새로운 데이터 예측
: predict 모듈사용

In [ ]:
model.predict(x_test)

###정리

- 단어 시퀀스는 이진 벡터로 인코딩될 수 있고 다른 인코딩 방식도 있음

- 과대적합 조심 -> 훈련세트이 이외의 데이터로 성능 모니터링

- 이진분류 문제 => 유닛 +  마지막에 sigmoid 활성화 함수를 가진 dense층으로 해야함 -> sigmoid 활성화 함수의 손실함수는 binary_crossentropy .
